# Get text from the last pickle to convert

In [2]:
import pandas as pd

In [37]:
import time

In [38]:
# My pickling function.
def datePickle(df):
    timestr = time.strftime("%Y%m%d-%H%M%S")
    df.to_pickle(f'{timestr}-df.pickle')
    print("Pickled")
    
# To summon pickle, use:
# df = pd.read_pickle('my_df.pickle')

In [6]:
df = pd.read_pickle('20230327-235802-df.pickle')
df.head()

,State,Abbreviation,Number,Summary,Bill Type,Date,Status,Erin Reed's State Risk,URL,Sponsors,Bill Title,Bill Intent
1,Alaska,AK,HB27,Designate Sex For School-sponsored Sports,Trans Sports Ban,1/19/2023,REFERRED TO EDUCATION,Moderate,https://legiscan.com/AK/bill/HB27/2023,Thomas McKay,"An Act relating to school athletics, recreatio...",WE WILL SHUT YOU OUT OF PUBLIC LIFE
2,Alaska,AK,SB96,Sex/reproduction Education; Schools,Don't Say Gay/Forced Outing,3/8/2023,REFERRED TO JUDICIARY,Moderate,https://legiscan.com/AK/bill/SB96/2023,Rules,An Act relating to parental rights in a child&...,WE WILL EMPOWER YOUR ENEMIES
3,Alaska,AK,HB105,Sex/reproduction Education; Schools,Don't Say Gay/Forced Outing,3/8/2023,REFERRED TO EDUCATION,Moderate,https://legiscan.com/AK/bill/HB105/2023,Rules,An Act relating to parental rights in a child&...,WE WILL EMPOWER YOUR ENEMIES
4,Arizona,AZ,SB1028,Adult cabaret performances; prohibited locations,Drag Ban,3/22/2023,House read second time,Low,https://legiscan.com/AZ/bill/SB1028/2023,Anthony Kern,Adult cabaret performances; prohibited locations,YOUR PRESENCE IS A PERVERSION
5,Arizona,AZ,SB1030,Sexually explicit performances; regulation,Drag Ban,3/21/2023,House read second time,Low,https://legiscan.com/AZ/bill/SB1030/2023,Anthony Kern,Sexually explicit performances; regulation,YOUR PRESENCE IS A PERVERSION


What am I doing? I'm trying to make a DF with the following columns:
'State', 'Sponsor', 'Bill Numbers'.

Try:
- Delete unneeded rows
- Flatten the Sponsors column
    - I think these are all String types, so I have to use the ", " to break them into individual list items
- Each Sponsor should have one State
- Turn all Bill Numbers corresponding to each Sponsor into a list

In [7]:
df = df.drop(['Abbreviation', 'Summary', 'Bill Type', 'Date', "Erin Reed's State Risk", 'Status', 'URL', 'Bill Title', 'Bill Intent'], axis=1)

In [8]:
df.head()

,State,Number,Sponsors
1,Alaska,HB27,Thomas McKay
2,Alaska,SB96,Rules
3,Alaska,HB105,Rules
4,Arizona,SB1028,Anthony Kern
5,Arizona,SB1030,Anthony Kern


In [17]:
# Check length and datatype
len(df)

453

In [16]:
type(df.at[400,'Sponsors'])

str

In [18]:
# Turn string into list

df['Sponsors'] = df['Sponsors'].str.split(', ')

In [19]:
# Check that it worked

type(df.at[400,'Sponsors'])

list

In [20]:
df.at[400,'Sponsors']

['Josh Hawley', 'Ted Cruz']

In [21]:
# Back up the df

df2 = df

In [22]:
# Flatten the sponsors so each list item becomes a new row

df = df.explode('Sponsors')

In [23]:
# Check that it worked 

df

,State,Number,Sponsors
1,Alaska,HB27,Thomas McKay
2,Alaska,SB96,Rules
3,Alaska,HB105,Rules
4,Arizona,SB1028,Anthony Kern
5,Arizona,SB1030,Anthony Kern
...,...,...,...
453,Wyoming,SF0133,John Kolb
453,Wyoming,SF0133,Troy McKeown
453,Wyoming,SF0133,Tim Salazar
453,Wyoming,SF0133,Wendy Davis Schuler


In [24]:
# Back up the df 

df2 = df

In [43]:
# Make a new df, sorting by sponsors and state 
# and aggregating remaining column into a list per row, 
# then turning that column into a list? Not 100% on this

dftest = df.groupby(['Sponsors','State']).agg(list).applymap(list)

In [44]:
# Check that it worked 

dftest.head()

,,Number
Sponsors,State,
Aaron Aylward,South Dakota,"[HB1116, HB1125, HB1208, HB1080]"
Aaron Bean,US,"[HB734, HB5]"
Aaron Freeman,Indiana,"[HB1407, SB0480]"
Aaron Pilkington,Arkansas,[SB294]
Abby Angelos,Wyoming,"[SF0159, SF0133]"


In [51]:
# Turn this weird double index back into columns

dftest = dftest.reset_index(level=['Sponsors', 'State'])

In [55]:
# Check that it worked

dftest

,Sponsors,State,Number
0,Aaron Aylward,South Dakota,"[HB1116, HB1125, HB1208, HB1080]"
1,Aaron Bean,US,"[HB734, HB5]"
2,Aaron Freeman,Indiana,"[HB1407, SB0480]"
3,Aaron Pilkington,Arkansas,[SB294]
4,Abby Angelos,Wyoming,"[SF0159, SF0133]"
...,...,...,...
1421,Wren Williams,Virginia,"[HB1387, HB1707, HB2432]"
1422,Zach Payne,Indiana,[HB1118]
1423,Zachary Nunn,US,[HB5]
1424,Zack Gramlich,Arkansas,[SB294]


In [23]:
dftest = pd.read_csv('curserinfo-20230407.csv')

In [29]:
dftest['Number'][0]

"['HB1116', 'HB1125', 'HB1208', 'HB1080']"

In [30]:
type(dftest['Number'][0].strip("[]").strip("''").split(", "))

list

In [31]:
dftest['Number'] = dftest['Number'].apply(lambda x: x.strip("[]").strip("''").split(", "))

In [33]:
type(dftest['Number'][0])

list

In [34]:
len(dftest['Number'][0])

4

In [39]:
datePickle(dftest)

Pickled


Adding a column with bill count.

Can also remove the Numbers.

In [40]:
dftest['Count'] = dftest['Number']

In [42]:
dftest['Count'] = dftest['Count'].apply(lambda x:len(x))

In [43]:
dftest.head()

,Sponsors,State,Number,Count
0,Aaron Aylward,South Dakota,"[HB1116', 'HB1125', 'HB1208', 'HB1080]",4
1,Aaron Bean,US,"[HB734', 'HB5]",2
2,Aaron Freeman,Indiana,"[HB1407', 'SB0480]",2
3,Aaron Pilkington,Arkansas,[SB294],1
4,Abby Angelos,Wyoming,"[SF0159', 'SF0133]",2


In [44]:
# Export to csv

dftest.to_csv('curserinfo-20230407-withcount.csv', index=False)

In [45]:
# Pickle again 

datePickle(dftest)

Pickled


In [54]:
len(dftest)

1426